# TensorFlow Lite Conversion

### Ref https://github.com/peteryuX/arcface-tf2

In [2]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
import tensorflow as tf
import os
import logging
import numpy as np
from pathlib import Path
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from layers import ArcMarginPenaltyLogists
from losses import softmax_loss
from glob import glob

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

1


In [4]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead'
backbone_type = 'MobileNetV2'
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
steps_per_epoch = dataset_len // batch_size
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

In [6]:
base_dir = "/hd/jbpark/models/checkpoints/w_tfidentity/"
save_name = "ms1m_check/SGD/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
best_checkpoint

'/hd/jbpark/models/checkpoints/w_tfidentity/ms1m_check/SGD/e_1_l_45.426002502441406.ckpt'

In [7]:
weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_45.tflite', 'wb') as f:
    f.write(tflite_model)

In [4]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='/hd/jbpark/models/tflite/ms1m_acc_45.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
input_details

[{'name': 'input_image',
  'index': 0,
  'shape': array([  1, 112, 112,   3], dtype=int32),
  'shape_signature': array([ -1, 112, 112,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [5]:
output_details = interpreter.get_output_details()
output_details

[{'name': 'Identity',
  'index': 181,
  'shape': array([  1, 512], dtype=int32),
  'shape_signature': array([ -1, 512], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]